In [51]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

from scipy.optimize import curve_fit

import gm2
import trfp
import plotting_functions as plt2
import analysis_helper as helper
import helper_function_candidates as helper_old

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
def __fp_moment(order, strength, x_pos, y_pos):
    """Returns the magnitude of a B-field given fp moment parameters
    and position.
    """
    
    if order == 0: return strength * x_pos ** 0
    elif order == 1: return strength * x_pos / 4.5 
    elif order == 2: return strength * y_pos / 4.5
    elif order == 3: return strength * x_pos * y_pos / 4.5**2
    elif order == 4: return strength * x_pos ** 2 / 4.5**2
    elif order == 5: return 0.5 * strength * x_pos ** 2 * y_pos / 4.5**3
    
def __multipole(order, skew, strength, x_pos, y_pos):
    """Returns the magnitude of a B-field given multipole parameters
    and position.
    """
    # Takes a multipole strength A normalized to 4.5 cm
    r_pos = np.sqrt(x_pos**2 + y_pos**2)
    theta = np.arctan2(y_pos, x_pos)
    if skew == 0:
        b_magnitude = strength * (r_pos/4.5)**order * np.cos(order*theta)
    if skew == 1:
        b_magnitude = strength * (r_pos/4.5)**order * np.sin(order*theta)
    return b_magnitude

FP6_X = np.array([-3, 0, 3, -3, 0, 3])
FP6_Y = np.array([7.7, 7.7, 7.7, -7.7, -7.7, -7.7])

_MOMENT_ORDER = [0, 1, 2, 3, 4, 5]

_MULTIPOLE_ORDER = [0, 1, 1, 2, 2]
_MULTIPOLE_SKEW = [0, 0, 1, 1, 0]

_MOMENTS = np.array([__multipole(_MULTIPOLE_ORDER[i], _MULTIPOLE_SKEW[i], 1, FP6_X, FP6_Y) for i in range(5)])
_MOMENTS[np.abs(_MOMENTS) < 1.0e-9] = 0 

THETA_FP6 = np.linalg.pinv(np.transpose(_MOMENTS))
THETA_FP6[np.abs(THETA_FP6) < 1.0e-9] = 0

In [48]:
print THETA_FP6

[[ 1.64694444 -2.79388889  1.64694444  1.64694444 -2.79388889  1.64694444]
 [-0.375       0.          0.375      -0.375       0.          0.375     ]
 [ 0.0974026   0.0974026   0.0974026  -0.0974026  -0.0974026  -0.0974026 ]
 [-0.10957792  0.          0.10957792  0.10957792  0.         -0.10957792]
 [ 0.5625     -1.125       0.5625      0.5625     -1.125       0.5625    ]]


In [50]:
print np.dot(trfp.J_6_PROBE, trfp.THETA_FP_6[:5])

[[ 1.64694448 -2.79388896  1.64694448  1.64694448 -2.79388896  1.64694448]
 [-0.375       0.          0.375      -0.375       0.          0.375     ]
 [ 0.0974026   0.0974026   0.0974026  -0.0974026  -0.0974026  -0.0974026 ]
 [-0.10957792  0.          0.10957792  0.10957792  0.         -0.10957792]
 [ 0.5625     -1.125       0.5625      0.5625     -1.125       0.5625    ]]


In [36]:
print np.dot(THETA_FP6, np.transpose(_MOMENTS))

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -1.03933216e-17 -1.10461584e-17  1.00000000e+00
   6.94890674e-18 -1.71775729e-19]
 [ 0.00000000e+00 -2.08166817e-17  5.55111512e-17  0.00000000e+00
   1.00000000e+00 -1.38777878e-17]
 [ 0.00000000e+00 -1.26762477e-17 -1.77187109e-17  2.51949602e-17
  -2.69545489e-17  1.00000000e+00]]


In [23]:
print np.dot(theta_test, np.transpose(theta_test))

[[ 1.66666667e-01 -3.46944695e-18  9.01155052e-19 -1.42682883e-19
   8.67361738e-18]
 [-3.46944695e-18  5.62500000e-01 -5.20417043e-18  3.46944695e-18
   0.00000000e+00]
 [ 9.01155052e-19 -5.20417043e-18  5.69235959e-02  1.64285247e-19
  -8.67361738e-19]
 [-1.42682883e-19  3.46944695e-18  1.64285247e-19  4.80292840e-02
  -1.73472348e-18]
 [ 8.67361738e-18  0.00000000e+00 -8.67361738e-19 -1.73472348e-18
   3.79687500e+00]]
